In [1]:
from datetime import datetime

import time
import os
import sys
import itertools
from pathlib import Path

import numpy as np
from scipy.stats import lognorm
import pandas as pd

from astropy import stats
from astropy.io import fits
from astropy.time import Time
import astropy.units as u

import matplotlib
#matplotlib.use('nbagg')
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

config = tf.ConfigProto(
    intra_op_parallelism_threads=8,
    inter_op_parallelism_threads=8,
    log_device_placement=True
)
sess = tf.Session(config=config)

np.random.seed(42)

print(tf.__version__)

1.14.0


In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

In [65]:
# load data and make some subsets for each wfs for inspection later
dfs = []
dfs.append(pd.read_csv(f"../raw_data/2017_wfs.csv"))
dfs.append(pd.read_csv(f"../raw_data/2018_2019_wfs_loose.csv"))
data = pd.concat(dfs)
data['ut'] = pd.to_datetime(data.ut)
data['az'][data['az'] < 0.] += 360.

f9 = data[(data['wfs'] == 'newf9') | (data['wfs'] == 'oldf9')]
f5 = data[data['wfs'] == 'f5']
mmirs = data[data['wfs'] == 'mmirs']
bino = data[data['wfs'] == 'binospec']

# wrangle the times to add colums for mjd to look for trends over time and hour to look for nightly trends
raw_times = data['time']
times = Time(raw_times.values.tolist(), format='isot', scale='utc')
mjd = times.mjd
data['mjd'] = mjd.tolist()
data['hour'] = data['ut'].dt.hour

e_series = pd.read_csv("../halcoll/data/halcoll_temps.csv")

fixed = data.drop(columns=['ut']).set_index(pd.DatetimeIndex(data['time'], name='ut'))

e_series = e_series.set_index(pd.DatetimeIndex(e_series['ts'], name='ut').tz_localize('MST').tz_convert(None)).drop(columns=['ts'])

merged = pd.merge_asof(fixed.sort_index(), e_series, on='ut')

# trim out columns not relevant to training
dropped_cols = [
    'ut', 'time', 'airmass', 'cc_x_err', 'cc_y_err', 'chamt', 'osst',
    'outt', 'exptime', 'file', 'focerr', 'fwhm', 'raw_seeing', 'residual_rms',
    'seeing', 'wavefront_rms', 'xcen', 'ycen', 'comaerr'
]
trimmed = merged.drop(columns=dropped_cols)
trimmed = trimmed.dropna()

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [66]:
def norm(x, s):
    return (x - s['mean']) / s['std']

labels = ['focus', 'tiltx', 'tilty', 'transx', 'transy']

# split data set up into three training sets: f9, mmirs, and f5/bino. these sets correspond to the three optical configurations
# we use: f/9 and no corrector, f/5 with no corrector (mmirs), and f/5 with spectroscopic corrector (f5/bino).
datasets = {}
wfs = trimmed.pop('wfs')
datasets['f9'] = trimmed[(wfs == 'newf9') | (wfs == 'oldf9')]
datasets['mmirs'] = trimmed[wfs == 'mmirs']
datasets['f5'] = trimmed[(wfs == 'f5') | (wfs == 'binospec')]

# the large range in offsets is messing up the training. so normalize the hexapod coords to their means
means = {}
train_datasets = {}
test_datasets = {}
train_labels = {}
test_labels = {}
train_stats = {}
normed_train_data = {}
normed_test_data = {}
for w in datasets:
    means[w] = {}
    for l in labels:
        means[w][l] = datasets[w][l].mean()
        datasets[w][l] -= means[w][l]

    train_datasets[w] = datasets[w].sample(frac=0.75, random_state=0)
    test_datasets[w] = datasets[w].drop(train_datasets[w].index)
    train_stats[w] = train_datasets[w].describe()
    train_stats[w] = train_stats[w].drop(columns=labels)
    train_stats[w] = train_stats[w].transpose()
    
    train_labels[w] = {}
    test_labels[w] = {}
    for l in labels:
        train_labels[w][l] = train_datasets[w].pop(l)
        test_labels[w][l] = test_datasets[w].pop(l)
        
    normed_train_data[w] = norm(train_datasets[w], train_stats[w])
    normed_test_data[w] = norm(test_datasets[w], train_stats[w])

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [67]:
def build_model(width=32, shape=1, nlayers=1, tolerance=0.001):
    nn_layers = [layers.Dense(width, activation=tf.nn.relu, input_shape=[shape])]  # input_shape=[len(train_dataset.keys())])]

    if nlayers > 0:
        for i in range(nlayers-1):
            nn_layers.append(layers.Dense(width, activation=tf.nn.relu))
    
    nn_layers.append(layers.Dense(1))
    model = keras.Sequential(nn_layers)
    optimizer = tf.keras.optimizers.RMSprop(tolerance)

    model.compile(
        loss='mean_squared_error',
        optimizer=optimizer,
        metrics=['mean_absolute_error', 'mean_squared_error']
    )
    return model

In [76]:
models = {}
mod_config = {
    "f9": {
        'width': 4,
        'nlayers': 4
    },
    "f5": {
        'width': 64,
        'nlayers': 2
    },
    'mmirs': {
        'width': 64,
        'nlayers': 2
    }
}

for w in datasets:
    models[w] = {}
    for l in labels:
        models[w][l] = build_model(
            width=mod_config[w]['width'],
            shape=len(train_datasets[w].keys()),
            nlayers=mod_config[w]['nlayers']
        )

# The patience parameter is the amount of epochs to check for improvement
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=400)

In [77]:
sel_wfs = 'f9'
models[sel_wfs]['focus'].summary()

Model: "sequential_150"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_490 (Dense)            (None, 4)                 80        
_________________________________________________________________
dense_491 (Dense)            (None, 4)                 20        
_________________________________________________________________
dense_492 (Dense)            (None, 4)                 20        
_________________________________________________________________
dense_493 (Dense)            (None, 4)                 20        
_________________________________________________________________
dense_494 (Dense)            (None, 1)                 5         
Total params: 145
Trainable params: 145
Non-trainable params: 0
_________________________________________________________________


In [78]:
# Display training progress by printing a single dot for each completed epoch
class PrintDot(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs):
        if epoch % 100 == 0: 
            print('')
        print('.', end='')

EPOCHS = 4000

histories = {}

for l in labels:
    print(f"Training {l} model....")
    histories[l] = models[sel_wfs][l].fit(
        normed_train_data[sel_wfs], train_labels[sel_wfs][l],
        epochs=EPOCHS, validation_split = 0.2, verbose=0,
        callbacks=[early_stop, PrintDot()]
    )
    print("\n")

Training focus model....

....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
.................................................................

In [79]:
for l in labels:
    models[sel_wfs][l].save(f"../halcoll/data/{sel_wfs}_{l}_2017_2018_2019_loose_4x4x4x4_model.h5")

In [80]:
def plot_history(history, label=None):
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch

    plt.figure()
    plt.xlabel('Epoch')
    plt.ylabel('Mean Abs Error')
    plt.plot(hist['epoch'], hist['mean_absolute_error'],
           label='Train Error')
    plt.plot(hist['epoch'], hist['val_mean_absolute_error'],
           label = 'Val Error')
    plt.legend()
    if label is not None:
        plt.title(label)

    plt.figure()
    plt.xlabel('Epoch')
    plt.ylabel('Mean Square Error')
    plt.plot(hist['epoch'], hist['mean_squared_error'],
           label='Train Error')
    plt.plot(hist['epoch'], hist['val_mean_squared_error'],
           label = 'Val Error')
    plt.legend()
    if label is not None:
        plt.title(label)
    plt.savefig(f"{label}_train.pdf")
    plt.show()

In [86]:
plot_history(histories['focus'], label="focus")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [82]:
def show_results(wfs, label):
    loss, mae, mse = models[wfs][label].evaluate(normed_test_data[wfs], test_labels[wfs][label], verbose=0)

    print("Testing set Mean Abs Error: {:5.2f} um".format(mae))
    print("Testing set RMS: {:5.2f} um".format(np.sqrt(mse)))

    test_predictions = models[wfs][label].predict(normed_test_data[wfs]).flatten()

    plt.figure()
    plt.scatter(test_labels[wfs][label], test_labels[wfs][label] - test_predictions)
    plt.xlabel('True Values [um]')
    plt.ylabel('Residuals [um]')
    minx, maxx = min(test_labels[wfs][label]), max(test_labels[wfs][label])
    plt.plot([minx, maxx], [0, 0])
    plt.show()

In [85]:
show_results(sel_wfs, 'focus')

Testing set Mean Abs Error: 73.90 um
Testing set RMS: 105.87 um


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [60]:
lab = 'focus'
test_predictions = models[sel_wfs][lab].predict(normed_test_data['f9']).flatten()
diff = test_labels[sel_wfs][lab] - test_predictions
diff.idxmax(), diff.idxmin()

(3040, 711)

In [ ]:
test_labels[sel_wfs][lab].idxmin()

In [61]:
test_datasets[sel_wfs].loc[711]

az                                        74.510000
el                                        36.940000
mjd                                    57780.184664
hour                                       4.000000
cell_e_series_backplate_C                  7.614150
cell_e_series_chamber_ambient_C           -1.385100
cell_e_series_frontplate_C                 6.334500
cell_e_series_in_front_of_primary_C        1.170600
cell_e_series_lower_plenum_C               7.882300
cell_e_series_midplate_C                   7.585700
cell_e_series_outside_ambient_C           -2.670350
yankee_temperature                        -2.294000
temptrax1_probe2                          -2.200000
temptrax1_probe3                          -1.800000
temptrax1_probe4                          -1.700000
temptrax1_probe6                          -1.900000
temptrax3_probe10                         -2.600000
temptrax3_probe11                         -2.300000
temptrax3_probe12                         -2.000000
Name: 711, d

In [ ]:
chk_data = normed_test_data[normed_test_data[sel_wfs] > 0]
#chk.index
chk_test = test_labels['focus'][chk_data.index] 
chk_pred = models['focus'].predict(chk_data).flatten()
chk_diff = chk_test - chk_pred
chk_diff.std()

In [ ]:
m = keras.models.load_model("/Users/tim/MMT/HALcoll/halcoll/data/tiltx_model.h5")

In [ ]:
m.evaluate(normed_test_data, test_labels['tiltx'])

In [ ]:
ttt = train_dataset['cell_e_series_chamber_ambient_C'] - train_dataset['yankee_temperature']
ttt.min()

In [14]:
len(trimmed)

42047

In [20]:
datasets['f9'].head()

,az,el,focus,tiltx,tilty,transx,transy,mjd,hour,cell_e_series_backplate_C,...,cell_e_series_midplate_C,cell_e_series_outside_ambient_C,yankee_temperature,temptrax1_probe2,temptrax1_probe3,temptrax1_probe4,temptrax1_probe6,temptrax3_probe10,temptrax3_probe11,temptrax3_probe12
1,186.873714,63.717322,-292.562965,-18.901051,-4.7069,17.398073,194.957709,57429.966957,23,10.7970,...,10.7028,19.5691,13.974,13.8,13.8,12.6,15.3,14.1,13.8,12.7
2,183.036610,53.286308,-187.312965,-12.331051,25.8031,289.208073,449.387709,57430.014040,0,11.0969,...,11.0770,21.3626,14.191,14.4,14.3,13.1,15.8,14.7,14.4,13.1
3,239.629020,41.280837,-419.142965,27.028949,7.9531,-19.201927,608.517709,57430.103669,2,10.9520,...,10.9224,10.1409,11.019,12.8,12.8,12.1,10.6,12.7,12.9,12.2
4,290.512635,51.553934,42.187035,-16.761051,32.7931,77.678073,519.577709,57430.152014,3,10.8139,...,10.7880,9.8597,10.755,10.8,11.6,11.4,9.6,10.9,11.7,11.4
5,290.593852,51.141871,16.057035,-22.241051,30.7331,110.388073,443.327709,57430.153450,3,10.8021,...,10.7709,9.9240,10.661,10.7,11.6,11.4,9.8,10.8,11.6,11.4


In [27]:
len(datasets['f9'])

239

In [84]:
plt.close('all')